In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)

## Lecture de l'ENC

In [ ]:
enc_ds = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc_ds.GetDriver().GetDescription() == 'S57' # S57
m_covr_lyr = enc_ds.GetLayerByName('M_COVR')
soundg_lyr = enc_ds.GetLayerByName('SOUNDG')

covr_geo = [f.GetGeometryRef().Clone() for f in m_covr_lyr if f.GetField('CATCOV') == 1][0]
covr_geo

### Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
json_drv = ogr.GetDriverByName('GeoJSON')
covr_path = 'data/covr.geojson'
def create_covr():
    silent_remove(covr_path)
    covr_ds = json_drv.CreateDataSource(covr_path)
    covr_lyr = covr_ds.CreateLayer('covr')
    covr_f = ogr.Feature(covr_lyr.GetLayerDefn())
    covr_f.SetGeometry(covr_geo)
    covr_lyr.CreateFeature(covr_f)
    covr_ds.Destroy()
create_covr()
del create_covr

## Lecture de la Bathy

In [ ]:
bathy_ds = gdal.Warp("data/bathy.tif", 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr'))
bathy_ds.FlushCache()

In [ ]:
# bathy_ds = gdal.Open('data/navd_bath_30m')
# assert bathy_ds.GetDriver().GetDescription() == 'AIG' # ArcInfo Binary Grid
assert bathy_ds.RasterCount == 1
bathy_band = bathy_ds.GetRasterBand(1)
bathy_np = bathy_band.ReadAsArray()
# bathy_np[bathy_np == bathy_band.GetNoDataValue()] = np.NaN

In [ ]:
bathy_np.shape

In [ ]:
geotransform = bathy_ds.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))